    Aplicamos la transformada de Burrows-Wheeler 
    

In [76]:
import io
import codecs

def insert_shift(string):
    auxlist = list(string)
    auxlist.insert(0, auxlist.pop())
    return "".join(auxlist)

def find_index(headquarter, string):
    try:
        print("indice: "+str(headquarter.index(string)))
        return headquarter.index(string)
    except ValueError:
        return -1


def obtain_bwt(headquarter):
    return "".join(string[-1] for string in headquarter)

def bwt(string):
    headquarter = list()
    aux=string
    for i in range(len(string)):
        headquarter.append(insert_shift(aux))
        aux = insert_shift(aux)
    headquarter.sort()
    output= (obtain_bwt(headquarter),find_index(headquarter,string))
    return output

def clean_text(bwt_text):
    return bwt_text

def bwt_from_bytes(filename):
    with open(filename, "rb") as f:
        data = f.read()
        clean_data =clean_text(codecs.decode(data[0:128], errors='ignore'))
        print(clean_data.encode('utf-8'))
        bwt_data = bwt(clean_data)
    return bwt_data


if __name__ == "__main__":
    filename = "imagen.jpg"
    bwt_data = bwt_from_bytes(filename)
    print(bwt_data)


b'\x12Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0c\x01\x00\x00\x03\x00\x00\x00\x01\x17p\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\r/\x00\x00\x01\x02\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x06\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x15\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\x01(\x00\x03\x00\x00\x00\x01\x00\x02'
indice: 105
('\x05\x05\x03\x03\x03\x03\x03\x03\x03\x01\x01\x03*\x00\x00\x00\x00\x00\x00p/\x00\x00\x02\x01\x00\x03\x00\x00\x00\x01\x00f\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x12\x06(\x15\x00\x01\x00\x02\x01\x1a\x1b\x00\x08M\x00\x00\x00\x00\x0c\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x01\x01\x02\x01\x01\x01\x01\x01\x00\r\x12M\x00ix\x17E', 105)


Aplicamos la destransformada de Burrows-Wheeler

In [77]:
import codecs

def insert_char(headquarters, string ):
    
    for i in range(len(headquarters)):
        auxlist = list(headquarters[i])
        auxlist.insert(0, string[i])
        headquarters[i]= "".join(auxlist)
    return headquarters
def sort_headquarters(headquarters):
    return sorted(headquarters)

def reverse_bwt(bwt_text):
    headquarters = list(bwt_text[0])
    while(len(headquarters)!=len(headquarters[0])):
        headquarters.sort()
        insert_char(headquarters,bwt_text[0])
    headquarters.sort()
    for i in range(len(headquarters)):
        if headquarters[i].startswith("las"):
            print(i)
    output = headquarters[bwt_text[1]]
    print(len(headquarters))
    return output

bwt_text = ('\x05\x05\x03\x03\x03\x03\x03\x03\x03\x01\x01\x03*\x00\x00\x00\x00\x00\x00p/\x00\x00\x02\x01\x00\x03\x00\x00\x00\x01\x00f\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x12\x06(\x15\x00\x01\x00\x02\x01\x1a\x1b\x00\x08M\x00\x00\x00\x00\x0c\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x01\x01\x02\x01\x01\x01\x01\x01\x00\r\x12M\x00ix\x17E', 105)

# Invertimos la cadena de texto.
output = reverse_bwt(bwt_text)

output = output.encode("utf-8")

print(output)

120
b'\x12Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0c\x01\x00\x00\x03\x00\x00\x00\x01\x17p\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\r/\x00\x00\x01\x02\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x06\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x15\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\x01(\x00\x03\x00\x00\x00\x01\x00\x02'
